In [24]:
import tensorflow as tf
import datetime, os
from model import *
from data import *

#### Unpack videos and labels

In [25]:
unpack_video('data/larynx/train', video_folder='video', image_folder='image', target_size=(256, 256))
unpack_tif('data/larynx/train', tif_folder='tifs', label_folder='label', target_size=(256, 256))

#### Launch TensorBoard Session

In [26]:
%load_ext tensorboard
%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


#### Initialize the model (U-Net)

In [27]:
model = unet()
model_checkpoint = ModelCheckpoint('unet_larynx.hdf5', monitor='loss', verbose=1, save_best_only=True)

#### Prepare training dataset

In [28]:
training_dataset = training_dataset_generator(2,
                                              'data/larynx/train',
                                              'image',
                                              'label',
                                              dict([]),
                                              save_to_dir=None)

In [30]:
# Set up TensorBoard callback
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=7, mode='auto', restore_best_weights=True)


# Train the model using the training dataset and TensorBoard callback
model.fit(training_dataset, 
          steps_per_epoch=10, 
          epochs=200, 
          callbacks=[model_checkpoint, tensorboard_callback])

Found 55 images belonging to 1 classes.
Found 55 images belonging to 1 classes.
Epoch 1/200
10/10 [==============================] - ETA: 0s - loss: 7.0844 - accuracy: 0.9065
Epoch 1: loss improved from inf to 7.08439, saving model to unet_larynx.hdf5


c:\Users\perseusf\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


10/10 [==============================] - 32s 3s/step - loss: 7.0844 - accuracy: 0.9065
Epoch 2/200
10/10 [==============================] - ETA: 0s - loss: 0.2196 - accuracy: 0.9642
Epoch 2: loss improved from 7.08439 to 0.21961, saving model to unet_larynx.hdf5
10/10 [==============================] - 21s 2s/step - loss: 0.2196 - accuracy: 0.9642
Epoch 3/200
10/10 [==============================] - ETA: 0s - loss: 0.1531 - accuracy: 0.9821
Epoch 3: loss improved from 0.21961 to 0.15314, saving model to unet_larynx.hdf5
10/10 [==============================] - 19s 2s/step - loss: 0.1531 - accuracy: 0.9821
Epoch 4/200
10/10 [==============================] - ETA: 0s - loss: 0.1555 - accuracy: 0.9759
Epoch 4: loss did not improve from 0.15314
10/10 [==============================] - 19s 2s/step - loss: 0.1555 - accuracy: 0.9759
Epoch 5/200
10/10 [==============================] - ETA: 0s - loss: 0.2655 - accuracy: 0.9580
Epoch 5: loss did not improve from 0.15314
10/10 [=================

KeyboardInterrupt: 

#### Testing the model

In [31]:
# testing the model
test_dataset = test_dataset_generator("data/larynx/test", num_image=5)
results = model.predict(test_dataset, 3, verbose=1)
saveResult("data/larynx/test", results)


5/5 [==============================] - 1s 176ms/step
